# QC

In [1]:
%%bash

#PAIRED READS QC

mkdir 3_qc
mkdir 4_merged_fastq
mkdir 5_merged_fasta

libraries=$(ls -1 2_demultiplexed)
for library in $libraries
do
    mkdir 3_qc/${library}/
    mkdir 4_merged_fastq/${library}/
    mkdir 5_merged_fasta/${library}/
    files=$(ls -1 2_demultiplexed/${library} | grep -v invalid | grep R1.fastq | sed 's/.R1.fastq.*//')
    for sample in $files
    do
        fastp -i 2_demultiplexed/${library}/${sample}.R1.fastq -I 2_demultiplexed/${library}/${sample}.R2.fastq \
        -o 3_qc/${library}/${sample}.R1.fastq -O 3_qc/${library}/${sample}.R2.fastq \
        --unpaired1 3_qc/${library}/${sample}_unpaired.R1.fastq \
        --unpaired2 3_qc/${library}/${sample}_unpaired.R2.fastq \
        --failed_out 3_qc/${library}/${sample}_failed.fastq \
        -q 30 \
        --cut_tail \
        --trim_front1 20 \
        --trim_front2 20 \
        --max_len1 106 \
        --max_len2 106 \
        -l 90 \
        --merge \
        --overlap_len_require 90 \
        --correction \
        --merged_out 4_merged_fastq/${library}/${sample}_merged.fastq \
        -w 6 \
        -h ./fastp_out/${sample}_fastp.html \
        -j ./fastp_out/${sample}_fastp.json
        cat 3_qc/${library}/${sample}.R1.fastq >> 4_merged_fastq/${library}/${sample}_merged.fastq
        cat 3_qc/${library}/${sample}_unpaired.R1.fastq >> 4_merged_fastq/${library}/${sample}_merged.fastq
        seqkit fq2fa 4_merged_fastq/${library}/${sample}_merged.fastq -o 5_merged_fasta/${library}/${sample}_merged.fasta
    done
done


Read1 before filtering:
total reads: 81
total bases: 6081
Q20 bases: 5138(84.4927%)
Q30 bases: 4471(73.5241%)

Read2 before filtering:
total reads: 81
total bases: 5326
Q20 bases: 4805(90.2178%)
Q30 bases: 4392(82.4634%)

Merged and filtered:
total reads: 5
total bases: 510
Q20 bases: 488(95.6863%)
Q30 bases: 461(90.3922%)

Filtering result:
reads passed filter: 10
reads failed due to low quality: 28
reads failed due to too many N: 0
reads failed due to too short: 124
reads with adapter trimmed: 10
bases trimmed due to adapters: 111
reads corrected by overlap analysis: 3
bases corrected by overlap analysis: 3

Duplication rate: 0%

Insert size peak (evaluated by paired-end reads): 143

Read pairs merged: 5
% of original read pairs: 6.17284%
% in reads after filtering: 100%


JSON report: ./fastp_out/21AU_fastp.json
HTML report: ./fastp_out/21AU_fastp.html

fastp -i 2_demultiplexed/ADD/21AU.R1.fastq -I 2_demultiplexed/ADD/21AU.R2.fastq -o 3_qc/ADD/21AU.R1.fastq -O 3_qc/ADD/21AU.R2.fastq

# derep - denoise - chimera - rerep

In [1]:
%%bash

#DEREPLICATE, DENOISE, CHIMERA DETECTION AND REREPLICATE

mkdir 6_denoise_uc/
mkdir 6_denoise_uc/derep/
mkdir 6_denoise_uc/denoise/
mkdir 6_denoise_uc/uchime/
mkdir 6_denoise_uc/rerep/
mkdir 6_denoise_uc/clusters/

libraries=$(ls -1 5_merged_fasta/)
for library in $libraries
do
    mkdir 6_denoise_uc/derep/${library}/
    mkdir 6_denoise_uc/denoise/${library}/
    mkdir 6_denoise_uc/uchime/${library}/
    mkdir 6_denoise_uc/rerep/${library}/
    mkdir 6_denoise_uc/clusters/${library}/
    
    files=$(ls -1 5_merged_fasta/${library} | grep _merged.fasta | sed 's/_merged.fasta*//')
    for sample in $files
    do
        
        vsearch --derep_fulllength 5_merged_fasta/${library}/${sample}_merged.fasta \
        --sizeout \
        --output 6_denoise_uc/derep/${library}/${sample}_derep.fasta --minuniquesize 3 \
        --uc 6_denoise_uc/clusters/${library}/${sample}_derep.txt
        
        vsearch --cluster_unoise 6_denoise_uc/derep/${library}/${sample}_derep.fasta \
        --sizein --sizeout --minsize 1 --unoise_alpha 2.0 \
        --id 0.99 \
        --centroids 6_denoise_uc/denoise/${library}/${sample}_denoise.fasta \
        --uc 6_denoise_uc/clusters/${library}/${sample}_denoise.txt
        
        vsearch --uchime_ref 6_denoise_uc/denoise/${library}/${sample}_denoise.fasta --db blast_db/12s_full.fasta \
        --chimeras 6_denoise_uc/uchime/${library}/${sample}_chimera.fasta \
        --borderline 6_denoise_uc/uchime/${library}/${sample}_chimera.fasta \
        --mindiffs 1 --mindiv 0.8 --nonchimeras 6_denoise_uc/uchime/${library}/${sample}_nc.fasta \
        
        vsearch --rereplicate 6_denoise_uc/uchime/${library}/${sample}_nc.fasta \
        --output 6_denoise_uc/rerep/${library}/${sample}_rerep.fasta #--uc stuff.txt
        
    done
done


vsearch v2.14.2_linux_x86_64, 31.3GB RAM, 12 cores
https://github.com/torognes/vsearch

Dereplicating file 5_merged_fasta/EA01/BLEL01_merged.fasta 100%
13272433 nt in 128542 seqs, min 90, max 121, avg 103
Sorting 100%
4993 unique sequences, avg cluster 25.7, median 1, max 38228
Writing output file 100%
Writing uc file, first part 100%
Writing uc file, second part 100%
1060 uniques written, 3933 clusters discarded (78.8%)
vsearch v2.14.2_linux_x86_64, 31.3GB RAM, 12 cores
https://github.com/torognes/vsearch

Reading file 6_denoise_uc/derep/EA01/BLEL01_derep.fasta 100%
109929 nt in 1060 seqs, min 91, max 120, avg 104
Masking 100%
Sorting by abundance 100%
Counting k-mers 100%
Clustering 100%
Sorting clusters 100%
Writing clusters 100%
Clusters: 182 Size min 3, max 46204, avg 5.8
Singletons: 0, 0.0% of seqs, 0.0% of clusters
vsearch v2.14.2_linux_x86_64, 31.3GB RAM, 12 cores
https://github.com/torognes/vsearch

Reading file blast_db/12s_full.fasta 100%
3656075 nt in 2259 seqs, min 63, max

# Blast

In [2]:
%%bash

mkdir blast_out

libraries=$(ls -1 6_denoise_uc/uchime/)
for library in $libraries
do
    mkdir blast_out/${library}/
    files=$(ls -1 6_denoise_uc/uchime/${library} | grep _nc.fasta | sed 's/_nc.fasta*//')
    for sample in $files
    do
        blastn -query 6_denoise_uc/uchime/${library}/${sample}_nc.fasta \
        -db blast_db/12s_full \
        -out blast_out/${library}/${sample}_blast.tsv \
        -perc_identity 98 \
        -outfmt '6 qseqid stitle sacc staxids pident qcovs evalue bitscore' \
        -num_alignments 20 \
        -num_threads 6
    done
done

# Taxonomy to Blast

In [3]:
%%bash

mkdir blast_tax

python tax_to_blast.py -i blast_out -o blast_tax -lin new_taxdump/rankedlineage.dmp


# LCA

In [4]:
%%bash

mkdir blast_lca

libraries=$(ls -1 blast_tax/)
for library in $libraries
do
    mkdir blast_lca/${library}/
    files=$(ls -1 blast_tax/${library} | grep _tax.tsv | sed 's/_tax.tsv*//')
    for sample in $files
    do
        python mlca.py -i blast_tax/${library}/${sample}_tax.tsv -o blast_lca/${library}/${sample}_lca.tsv \
        -b 10 -id 100 -cov 60 -m 100 -hits 1
    done
done

# LCA to TSV

In [5]:
!python mlca-tsv.py -i blast_lca/ -r 6_denoise_uc/rerep/  -o final_out.tsv